# Parcels

In [1]:
# import random
# random_values = random.sample(range(100), 9)
# print(random_values)
# ensemble members I will run: [0, 65, 32, 85, 61, 90, 80, 68, 73, 49]

### Import statements

In [2]:
import parcels
from parcels import AdvectionRK4_3D,FieldSet, Field, ParticleSet, Variable, JITParticle, StatusCode
print(f"Parcels version: {parcels.__version__}")  # Should be 3.1.3 or higher

Parcels version: 3.1.4


In [3]:
for statuscode, val in StatusCode.__dict__.items():
    if statuscode.startswith("__"):
        continue
    print(f"{statuscode} = {val}")

def CheckOutOfBounds(particle, fieldset, time):
    if particle.state == StatusCode.ErrorOutOfBounds:
        particle.delete()

def DeleteParticle(particle, fieldset, time):
    particle.delete()
        
def CheckError(particle, fieldset, time):
    if particle.state >= 50:  # This captures all Errors
        particle.delete()

def KeepInOcean(particle, fieldset, time):
    if particle.state == StatusCode.ErrorThroughSurface:
        particle_ddepth = 0.0
        particle.state = StatusCode.Success

Success = 0
Evaluate = 10
Repeat = 20
Delete = 30
StopExecution = 40
StopAllExecution = 41
Error = 50
ErrorInterpolation = 51
ErrorOutOfBounds = 60
ErrorThroughSurface = 61
ErrorTimeExtrapolation = 70


In [4]:
import cesm2_lens_utils

In [5]:
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
from xgcm import Grid
import tempfile
import pop_tools
import os
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.collections as mcollections
import cmocean
import cftime

### Functions

In [6]:
def ocn_var_ens(ens_ind, VAR):
    COMP = 'ocn'
    DIRECTORY = f'/glade/campaign/cgd/cesm/CESM2-LE/{COMP}/proc/tseries/month_1/{VAR}/'
    ds_var_hist_var, ds_var_fut_var = cesm2_lens_utils.get_ds_var(
        directory=DIRECTORY, var=VAR, comp=COMP, index_hist = ens_ind)
    var_ds = ds_var_hist_var[VAR].sel(time=slice('1958-01', '2015-01'))
    return var_ds

In [7]:
def atm_var_ens(ens_ind, VAR):
    COMP = 'atm'
    DIRECTORY = f'/glade/campaign/cgd/cesm/CESM2-LE/{COMP}/proc/tseries/month_1/{VAR}/'
    ds_var_hist_var, ds_var_fut_var = cesm2_lens_utils.get_ds_var(
        directory=DIRECTORY, var=VAR, comp=COMP, index_hist = ens_ind)
    var_ds = ds_var_hist_var[VAR].sel(time=slice('1958-01', '2015-01')).compute()
    return var_ds

In [8]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [9]:
def regrid_wvel_z_t(w_ds, grid_ds):
    grid_ds = xr.open_dataset(grid_file)
    coord_ds = xr.Dataset({
        'z_t': grid_ds.z_t,
        'z_w_top': w_ds.z_w_top
    })
    
    grid = Grid(coord_ds,
               coords={'Z': {'center': 'z_t', 'outer': 'z_w_top'}},
               periodic=False,
               autoparse_metadata=False,
               boundary={'Z': 'fill'}) 
    
    w_on_zt = grid.interp(w_ds, 'Z', boundary='fill')
    return w_on_zt

In [10]:
def create_pset(depth_val, lon_min, lon_max, lat_min, lat_max):
    lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 20),
                                    np.linspace(lat_min, lat_max, 20))
    lons = lon_grid.flatten();lats = lat_grid.flatten()
    pset = ParticleSet.from_list(fieldset=fieldset, 
                                pclass=JITParticle,
                                lon=lons,
                                lat=lats,
                                depth=np.ones(400)*depth_val)
    return pset

## Parcel trajectory
https://github.com/Parcels-code/Parcels/blob/v4-dev/src/parcels/_datasets/structured/circulation_models.py


In [11]:
# grid = pop_tools.get_grid('POP_gx1v7').isel(z_t = slice(0, 27))
grid_file = 'POP_grid_temp.nc'
# grid.to_netcdf(grid_file)

### FOSI

In [18]:
firstyear = 1959
lastyear = 2020 #2020
grid = pop_tools.get_grid('POP_gx1v7')
mask = xr.where((grid['REGION_MASK']>0) & (grid['REGION_MASK']<9), 1, np.nan)
fpath = '/glade/campaign/cesm/development/espwg/SMYLE/initial_conditions/SMYLE-FOSI/ocn/proc/tseries/month_1/'
fosi_montime_vals = [
    cftime.DatetimeNoLeap(1958+year, 1+month, 15) for year in range(63) for month in range(12)]#[-240-60:-60]#[-240:]#[0:240]

In [30]:
field = 'VVEL'
fname = f'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.pop.h.{field}.030601-036812.nc'
# ds_smyle_fosi_vvel = xr.open_dataset(fpath+fname)[field].isel(z_t = slice(0,27), time=slice(0, 240)).compute()
ds_smyle_fosi_vvel = xr.open_dataset(fpath+fname)[field].isel(z_t = slice(0,27), time=slice(456,696)).compute()
ds_smyle_fosi_vvel['time'] = fosi_montime_vals

field = 'UVEL'
fname = f'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.pop.h.{field}.030601-036812.nc'
# ds_smyle_fosi_uvel = xr.open_dataset(fpath+fname)[field].isel(z_t = slice(0,27), time=slice(0, 240)).compute()
ds_smyle_fosi_uvel = xr.open_dataset(fpath+fname)[field].isel(z_t = slice(0,27), time=slice(456, 696)).compute()

ds_smyle_fosi_uvel['time'] = fosi_montime_vals

field = 'WVEL'
fname = f'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.pop.h.{field}.030601-036812.nc'
# ds_smyle_fosi_wvel = xr.open_dataset(fpath+fname)[field].isel(z_w_top = slice(0,28), time=slice(0, 240)).compute()
ds_smyle_fosi_wvel = xr.open_dataset(fpath+fname)[field].isel(z_w_top = slice(0,28), time=slice(456, 696)).compute()
ds_smyle_fosi_wvel['time'] = fosi_montime_vals

# grid_ds = xr.open_dataset(grid_file)
# fosi_w_on_zt = regrid_wvel_z_t(ds_smyle_fosi_wvel, grid_ds)
# fosi_w_zt_file = '/glade/derecho/scratch/cassiacai/fosi_wvel_clean_1959_1980.nc'
# fosi_w_on_zt.to_netcdf(fosi_w_zt_file)

grid_ds = xr.open_dataset(grid_file)
fosi_w_on_zt = regrid_wvel_z_t(ds_smyle_fosi_wvel, grid_ds)
fosi_w_zt_file = '/glade/derecho/scratch/cassiacai/fosi_wvel_clean_1996_2015.nc'
fosi_w_on_zt.to_netcdf(fosi_w_zt_file)

In [32]:
# fosi_uvel_file = '/glade/derecho/scratch/cassiacai/fosi_uvel_1996_2015.nc'
# fosi_vvel_file = '/glade/derecho/scratch/cassiacai/fosi_vvel_1996_2015.nc'

# ds_smyle_fosi_uvel.to_netcdf(fosi_uvel_file)
# ds_smyle_fosi_vvel.to_netcdf(fosi_vvel_file)

In [28]:
fosi_uvel_file = '/glade/derecho/scratch/cassiacai/fosi_uvel_1959_1980.nc'
fosi_vvel_file = '/glade/derecho/scratch/cassiacai/fosi_vvel_1959_1980.nc'
fosi_w_zt_file = '/glade/derecho/scratch/cassiacai/fosi_wvel_clean_1959_1980.nc'

In [29]:
month = 11
ds_original_uvel = xr.open_dataset(fosi_uvel_file)
ds_temp_uvel = ds_original_uvel.isel(time=slice(month, None))

ds_original_vvel = xr.open_dataset(fosi_vvel_file)
ds_temp_vvel = ds_original_vvel.isel(time=slice(month, None))

ds_original_z_t = xr.open_dataset(fosi_w_zt_file)
ds_temp_zt = ds_original_z_t.isel(time=slice(month, None))

In [30]:
temp_uvel_output_file = '/glade/derecho/scratch/cassiacai/fosi_uvel_temp_timestep{}_90.nc'.format(month)
ds_temp_uvel.to_netcdf(temp_uvel_output_file)

temp_vvel_output_file = '/glade/derecho/scratch/cassiacai/fosi_vvel_temp_timestep{}_90.nc'.format(month)
ds_temp_vvel.to_netcdf(temp_vvel_output_file)

temp_zt_output_file = '/glade/derecho/scratch/cassiacai/fosi_z_t_temp_timestep{}_90.nc'.format(month)
ds_temp_zt.to_netcdf(temp_zt_output_file)

In [12]:
month = 11
temp_uvel_output_file = '/glade/derecho/scratch/cassiacai/fosi_uvel_temp_timestep{}_90.nc'.format(month)
# ds_temp_uvel.to_netcdf(temp_uvel_output_file)

temp_vvel_output_file = '/glade/derecho/scratch/cassiacai/fosi_vvel_temp_timestep{}_90.nc'.format(month)
# ds_temp_vvel.to_netcdf(temp_vvel_output_file)

temp_zt_output_file = '/glade/derecho/scratch/cassiacai/fosi_z_t_temp_timestep{}_90.nc'.format(month)
# ds_temp_zt.to_netcdf(temp_zt_output_file)

In [10]:
# fosi_uvel_file = '/glade/derecho/scratch/cassiacai/fosi_uvel_1996_2015.nc'
# fosi_vvel_file = '/glade/derecho/scratch/cassiacai/fosi_vvel_1996_2015.nc'
# fosi_w_zt_file = '/glade/derecho/scratch/cassiacai/fosi_wvel_clean_1996_2015.nc'

### LENS

In [ ]:
###### ENS_IND_LS
temp_dir_ls = []
ENS_IND_LS = [32, 85, 61, 90, 80, 68, 73, 49] #[0, 65  32, 85, 61, 90, 80, 68, 73, 49]
for ENS_MEMB in ENS_IND_LS:
    print(ENS_MEMB)
    LENS_UVEL = ocn_var_ens(ENS_MEMB, 'UVEL').isel(time=slice(0,240), z_t =slice(0, 27)).compute()
    LENS_VVEL = ocn_var_ens(ENS_MEMB, 'VVEL').isel(time=slice(0,240), z_t = slice(0, 27)).compute()
    LENS_WVEL = ocn_var_ens(ENS_MEMB, 'WVEL').isel(time=slice(0,240), z_w_top=slice(0, 28)).compute()
    
    LENS_UVEL = ocn_var_ens(ENS_MEMB, 'UVEL').isel(time=slice(456,696), z_t =slice(0, 27)).compute()
    LENS_VVEL = ocn_var_ens(ENS_MEMB, 'VVEL').isel(time=slice(456,696), z_t = slice(0, 27)).compute()
    LENS_WVEL = ocn_var_ens(ENS_MEMB, 'WVEL').isel(time=slice(456,696), z_w_top=slice(0, 28)).compute()
    
    temp_dir = tempfile.mkdtemp()
    temp_dir_ls.append(temp_dir)
    print(temp_dir)
    uvel_file = os.path.join(temp_dir, 'uvel_temp.nc')
    vvel_file = os.path.join(temp_dir, 'vvel_temp.nc') 
    
    LENS_UVEL.to_netcdf(uvel_file)
    LENS_VVEL.to_netcdf(vvel_file)

    grid_file = 'POP_grid_temp.nc'
    grid_ds = xr.open_dataset(grid_file)
    w_on_zt = regrid_wvel_z_t(LENS_WVEL, grid_ds)
    w_zt_file = '/glade/derecho/scratch/cassiacai/wvel_clean_{}.nc'.format(ENS_MEMB)
    w_on_zt.to_netcdf(w_zt_file)

    uvel_file = '{}/uvel_temp.nc'.format(temp_dir)
    vvel_file = '{}/vvel_temp.nc'.format(temp_dir)
    wvel_file = '{}/wvel_temp.nc'.format(temp_dir)
    w_zt_file = '/glade/derecho/scratch/cassiacai/wvel_clean_{}.nc'.format(ENS_MEMB)

    filenames_corrected = {
        'U': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': uvel_file},
        'V': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': vvel_file},
        'W': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': w_zt_file}
}

    variables = {'U': 'UVEL',
                 'V': 'VVEL',
                 'W': 'WVEL'}
    
    dimensions = {'U': {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'},
                  'V': {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'},
                  'W':  {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'}}

    fieldset = FieldSet.from_pop(filenames_corrected, variables, dimensions,
                            allow_time_extrapolation=True, deferred_load=False,
                            depth_units='cm')

    ########### NORTH PACIFIC
    lon_min, lon_max = 180, 280; lat_min, lat_max = -50, -10
    
    DEPTH_INIT=500
    pset = create_pset(DEPTH_INIT, lon_min, lon_max, lat_min, lat_max)
    output_file = pset.ParticleFile(
        name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_SH_startingat{}m_1958_1977".format(
        ENS_MEMB, DEPTH_INIT), 
        outputdt=timedelta(days=30))
    pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],
                 runtime=timedelta(days=365*20),
                 dt=timedelta(days=2),output_file=output_file)
    print("Simulation completed!")
    
    DEPTH_INIT=5000
    pset = create_pset(DEPTH_INIT, lon_min, lon_max, lat_min, lat_max)
    output_file = pset.ParticleFile(
        name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_SH_startingat{}m_1958_1977".format(
        ENS_MEMB, DEPTH_INIT), 
        outputdt=timedelta(days=30))
    pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],
                 runtime=timedelta(days=365*20),
                 dt=timedelta(days=2),output_file=output_file)
    print("Simulation completed!")
    
    ########### NORTH PACIFIC
    lon_min, lon_max = 150, 230; lat_min, lat_max = 10, 45

    DEPTH_INIT=500
    pset = create_pset(DEPTH_INIT, lon_min, lon_max, lat_min, lat_max)
    output_file = pset.ParticleFile(
        name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_NH_startingat{}m_1958_1977".format(
        ENS_MEMB, DEPTH_INIT), 
        outputdt=timedelta(days=30))
    pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],
                 runtime=timedelta(days=365*20),
                 dt=timedelta(days=2),output_file=output_file)
    print("Simulation completed!")
    
    DEPTH_INIT=5000
    pset = create_pset(DEPTH_INIT, lon_min, lon_max, lat_min, lat_max)
    output_file = pset.ParticleFile(
        name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_NH_startingat{}m_1958_1977".format(
        ENS_MEMB, DEPTH_INIT), 
        outputdt=timedelta(days=30))
    pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],
                 runtime=timedelta(days=365*20),
                 dt=timedelta(days=2),output_file=output_file)
    print("Simulation completed!")

32
/glade/derecho/scratch/cassiacai/tmp/tmptg8ew7a5


In [12]:
# ENS_IND_LS = [0, 65  32, 85, 61, 90, 80, 68, 73, 49] # 0, 90, 65, 49, 32, 85, 80
# ENS_IND_LS = [68, 73] # 0, 90, 65, 49, 32, 85, 61, 80

ENS_MEMB = 90

LENS_UVEL = ocn_var_ens(ENS_MEMB, 'UVEL').isel(time=slice(0,240), z_t =slice(0, 27)).compute()
LENS_VVEL = ocn_var_ens(ENS_MEMB, 'VVEL').isel(time=slice(0,240), z_t = slice(0, 27)).compute()
LENS_WVEL = ocn_var_ens(ENS_MEMB, 'WVEL').isel(time=slice(0,240), z_w_top=slice(0, 28)).compute()

# LENS_UVEL = ocn_var_ens(ENS_MEMB, 'UVEL').isel(time=slice(456,696), z_t =slice(0, 27)).compute()
# LENS_VVEL = ocn_var_ens(ENS_MEMB, 'VVEL').isel(time=slice(456,696), z_t = slice(0, 27)).compute()
# LENS_WVEL = ocn_var_ens(ENS_MEMB, 'WVEL').isel(time=slice(456,696), z_w_top=slice(0, 28)).compute()

temp_dir = tempfile.mkdtemp()

uvel_file = os.path.join(temp_dir, 'uvel_temp.nc')
vvel_file = os.path.join(temp_dir, 'vvel_temp.nc') 

LENS_UVEL.to_netcdf(uvel_file)
LENS_VVEL.to_netcdf(vvel_file)

grid_ds = xr.open_dataset(grid_file)
w_on_zt = regrid_wvel_z_t(LENS_WVEL, grid_ds)
w_zt_file = '/glade/derecho/scratch/cassiacai/wvel_clean_{}.nc'.format(ENS_MEMB)
w_on_zt.to_netcdf(w_zt_file)

In [13]:
print(temp_dir)

/glade/derecho/scratch/cassiacai/tmp/tmpsdz_m67_


In [ ]:
# '/glade/derecho/scratch/cassiacai/tmp/tmp1zpy_6bb' --> 65 1998
# '/glade/derecho/scratch/cassiacai/tmp/tmp1zpy_6bb' --> 32 1998
# '/glade/derecho/scratch/cassiacai/tmp/tmp1zpy_6bb' --> 85 1998


In [18]:
#### SCRATCH FOR REGRIDDING WVEL TO Z_T
# w_ds = LENS_WVEL
# grid_ds = xr.open_dataset(grid_file)

# coord_ds = xr.Dataset({
#     'z_t': grid_ds.z_t,
#     'z_w_top': w_ds.z_w_top
# })

# grid = Grid(coord_ds,
#            coords={'Z': {'center': 'z_t', 'outer': 'z_w_top'}},
#            periodic=False,
#            autoparse_metadata=False,
#            boundary={'Z': 'fill'})  # Add boundary handling

# w_on_zt = grid.interp(w_ds, 'Z', boundary='fill')
# w_on_zt.to_netcdf('/glade/derecho/scratch/cassiacai/wvel_clean.nc')

In [12]:
ENS_MEMB = 90
uvel_file = '/glade/derecho/scratch/cassiacai/tmp/tmpsdz_m67_/uvel_temp.nc'
vvel_file = '/glade/derecho/scratch/cassiacai/tmp/tmpsdz_m67_/vvel_temp.nc'
# wvel_file = '/glade/derecho/scratch/cassiacai/tmp/tmpsdz_m67_/wvel_temp.nc'
w_zt_file = '/glade/derecho/scratch/cassiacai/wvel_clean_{}.nc'.format(ENS_MEMB)

In [13]:
month = 11
ds_original_uvel = xr.open_dataset(uvel_file)
ds_temp_uvel = ds_original_uvel.isel(time=slice(month, None))

ds_original_vvel = xr.open_dataset(vvel_file)
ds_temp_vvel = ds_original_vvel.isel(time=slice(month, None))

ds_original_z_t = xr.open_dataset(w_zt_file)
ds_temp_zt = ds_original_z_t.isel(time=slice(month, None))

In [13]:
month = 11
temp_uvel_output_file = '/glade/derecho/scratch/cassiacai/uvel_temp_timestep{}_90.nc'.format(month)
# ds_temp_uvel.to_netcdf(temp_uvel_output_file)

temp_vvel_output_file = '/glade/derecho/scratch/cassiacai/vvel_temp_timestep{}_90.nc'.format(month)
# ds_temp_vvel.to_netcdf(temp_vvel_output_file)

temp_zt_output_file = '/glade/derecho/scratch/cassiacai/z_t_temp_timestep{}_90.nc'.format(month)
# ds_temp_zt.to_netcdf(temp_zt_output_file)

In [14]:
### 
# 0 ---> tmpimapci25 # done with equatorial thermocline
# 65 ---> tmpxnqqjhnu # done with equatorial thermocline
# 32 ---> tmpl3obrs3d # done with equatorial thermocline
# 85 ---> tmpt8iax72l # done with equatorial thermocline
# 49  ---> tmp1dle1t7_ (1958 - 1977) # done with equatorial thermocline
# 61 ---> tmpu4ucdn3c (1958 - 1977) # done with equatorial thermocline
# 80 ---> tmpk9jwhxbu (1958 - 1977) # done with equatorial thermocline
# 68 ---> tmpmco3g6n0 (1958 - 1977) # done with equatorial thermocline
# 73 ---> tmpi43qza46 (1958 - 1977) # done with equatorial thermocline
# 90 ---> tmpsdz_m67_ 

### Variables

In [13]:
fosi_uvel_file = temp_uvel_output_file
fosi_vvel_file = temp_vvel_output_file
fosi_w_zt_file = temp_zt_output_file

In [14]:
# FOR FOSI
# Use the corrected W file
filenames_corrected = {
    'U': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': fosi_uvel_file},
    'V': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': fosi_vvel_file},
    'W': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': fosi_w_zt_file}
}

variables = {'U': 'UVEL',
             'V': 'VVEL',
             'W': 'WVEL'}

dimensions = {'U': {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'},
              'V': {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'},
              'W':  {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'}}

In [16]:
uvel_file = temp_uvel_output_file
vvel_file = temp_vvel_output_file
w_zt_file = temp_zt_output_file

In [17]:
# FOR LENS
# Use the corrected W file
filenames_corrected = {
    'U': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': uvel_file},
    'V': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': vvel_file},
    'W': {'lon': grid_file, 'lat': grid_file, 'depth': grid_file, 'data': w_zt_file}
}

variables = {'U': 'UVEL',
             'V': 'VVEL',
             'W': 'WVEL'}

dimensions = {'U': {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'},
              'V': {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'},
              'W':  {'lon': 'ULONG', 'lat': 'ULAT', 'depth': 'z_t', 'time': 'time'}}

In [15]:
%%time
fieldset = FieldSet.from_pop(filenames_corrected, variables, dimensions,
                            allow_time_extrapolation=True, deferred_load=False,
                            depth_units='cm')

CPU times: user 22.6 s, sys: 7.14 s, total: 29.7 s
Wall time: 35 s


# Southern Hemisphere subtropics

In [16]:
depth_val = 5000 # 500
lon_min, lon_max = 180, 280
lat_min, lat_max = -50, -10

lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 20),
                                np.linspace(lat_min, lat_max, 20))

lons = lon_grid.flatten()
lats = lat_grid.flatten()

pset = ParticleSet.from_list(fieldset=fieldset, 
                            pclass=JITParticle,
                            lon=lons,
                            lat=lats,
                            depth=np.ones(400)*depth_val)

In [16]:
# output_file = pset.ParticleFile(
#     name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_SH_startingat50m_1958_1977".format(ENS_MEMB), 
#                                outputdt=timedelta(days=30))

In [20]:
# output_file = pset.ParticleFile(
#     name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens90_month{}_SH_startingat50m_1958_1977".format(month), 
#                                outputdt=timedelta(days=30))

In [17]:
output_file = pset.ParticleFile(
    name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_fosi_month{}_SH_startingat50m_1958_1977".format(month), 
                               outputdt=timedelta(days=30))

In [18]:
# Run the particle simulation
print("Starting particle simulation...")
pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],                    # 4th-order Runge-Kutta advection
             runtime=timedelta(days=365*20),      # Run for 7 days
             dt=timedelta(days=2),       # Time step of 2 days
             output_file=output_file,        # Save trajectories
             
            )

print("Simulation completed!")

Starting particle simulation...
INFO: Output files are stored in /glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_fosi_month11_SH_startingat50m_1958_1977.zarr.
 38%|███▊      | 238636800.0/630720000.0 [04:31<09:05, 719168.91it/s] Correct cell not found for (lat, lon) = (-42.513819, 320.067347) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 68 318
            new particle indices: (yi, xi) 68 0
            Mesh 2d shape:  384 320
            Relative particle position:  (eta, xsi) 2.0731282663201311e-01 1.0598636104259640e+00
Correct cell not found for (lat, lon) = (-42.503612, 320.147967) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 68 318
            new particle indices: (yi, xi) 68 0
            Mesh 2d shape:  384 320
            Relative particle position:  (eta, xsi) 2.2641729820789866e-01 1.1315259576076642e+00
Correct cell not found for (lat, lon) = (-42.516066, 320.011714) after 1000000 iterations
Debug in

In [18]:
depth_val = 500
lon_min, lon_max = 180, 280
lat_min, lat_max = -50, -10

lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 20),
                                np.linspace(lat_min, lat_max, 20))

lons = lon_grid.flatten()
lats = lat_grid.flatten()

pset = ParticleSet.from_list(fieldset=fieldset, 
                            pclass=JITParticle,
                            lon=lons,
                            lat=lats,
                            depth=np.ones(400)*depth_val)

In [19]:
output_file = pset.ParticleFile(
    name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_SH_startingat5m_1958_1977".format(ENS_MEMB), 
                               outputdt=timedelta(days=30))

In [20]:
# Run the particle simulation
print("Starting particle simulation...")
pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],                    # 4th-order Runge-Kutta advection
             runtime=timedelta(days=365*20),      # Run for 7 days
             dt=timedelta(days=2),       # Time step of 2 days
             output_file=output_file,        # Save trajectories
             
            )

print("Simulation completed!")

Starting particle simulation...
INFO: Output files are stored in /glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens73_SH_startingat5m_1958_1977.zarr.
100%|██████████| 630720000.0/630720000.0 [12:01<00:00, 874351.94it/s] 
Simulation completed!


# Northern Hemisphere subtropics

In [21]:
depth_val = 500 # 5000
lon_min, lon_max = 150, 230
lat_min, lat_max = 10, 45

lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 20),
                                np.linspace(lat_min, lat_max, 20))

lons = lon_grid.flatten()
lats = lat_grid.flatten()

pset = ParticleSet.from_list(fieldset=fieldset, 
                            pclass=JITParticle,
                            lon=lons,
                            lat=lats,
                            depth=np.ones(400)*depth_val)

In [22]:
output_file = pset.ParticleFile(
    name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_NH_startingat5m_1958_1977".format(ENS_MEMB), 
                               outputdt=timedelta(days=30))

In [23]:
# Run the particle simulation
print("Starting particle simulation...")
pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],                    # 4th-order Runge-Kutta advection
             runtime=timedelta(days=365*20),      # Run for 7 days
             dt=timedelta(days=2),       # Time step of 2 days
             output_file=output_file,        # Save trajectories
             
            )

print("Simulation completed!")

Starting particle simulation...
INFO: Output files are stored in /glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens73_NH_startingat5m_1958_1977.zarr.
100%|██████████| 630720000.0/630720000.0 [12:19<00:00, 853174.89it/s] 
Simulation completed!


In [15]:
depth_val = 5000 # 5000
lon_min, lon_max = 150, 230
lat_min, lat_max = 10, 45

lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 20),
                                np.linspace(lat_min, lat_max, 20))

lons = lon_grid.flatten()
lats = lat_grid.flatten()

pset = ParticleSet.from_list(fieldset=fieldset, 
                            pclass=JITParticle,
                            lon=lons,
                            lat=lats,
                            depth=np.ones(400)*depth_val)

In [16]:
output_file = pset.ParticleFile(
    name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_NH_startingat50m_1958_1977".format(ENS_MEMB), 
                               outputdt=timedelta(days=30))

In [17]:
# Run the particle simulation
print("Starting particle simulation...")
pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],                    # 4th-order Runge-Kutta advection
             runtime=timedelta(days=365*20),      # Run for 7 days
             dt=timedelta(days=2),       # Time step of 2 days
             output_file=output_file,        # Save trajectories
             
            )

print("Simulation completed!")

Starting particle simulation...
INFO: Output files are stored in /glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens73_NH_startingat50m_1958_1977.zarr.
100%|██████████| 630720000.0/630720000.0 [13:16<00:00, 791624.72it/s] 
Simulation completed!


# Equatorial thermocline

In [16]:
# lon_min, lon_max = 140, 280
# lat_min, lat_max = -1, 1

# lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 20),
#                                 np.linspace(lat_min, lat_max, 20))

# lons = lon_grid.flatten()
# lats = lat_grid.flatten()

# # FOSI
# lats = [-0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476]
# lons = [140.56250391150786, 141.68750394281383, 142.8125039741198, 143.93750400542578, 145.06250403673175, 146.18750406803773, 147.3125040993437, 148.4375041306497, 149.56250416195567, 150.68750419326165, 151.81250422456762, 152.9375042558736, 154.06250428717956, 155.18750431848554, 156.31250434979154, 157.4375043810975, 158.56250441240348, 159.68750444370946, 160.81250447501543, 161.9375045063214, 163.06250453762738, 164.18750456893338, 165.31250460023935, 166.43750463154532, 167.5625046628513, 168.68750469415727, 169.81250472546324, 170.93750475676921, 172.06250478807522, 173.1875048193812, 174.31250485068716, 175.43750488199314, 176.5625049132991, 177.68750494460508, 178.81250497591105, 179.93750500721706, 181.062505038523, 182.18750506982897, 183.31250510113495, 184.43750513244092, 185.5625051637469, 186.6875051950529, 187.81250522635887, 188.93750525766484, 190.0625052889708, 191.18750532027678, 192.3125053515828, 193.43750538288876, 194.56250541419473, 195.68750544550073, 196.81250547680668, 197.93750550811265, 199.06250553941862, 200.1875055707246, 201.3125056020306, 202.4375056333366, 203.56250566464254, 204.68750569594854, 205.8125057272545, 206.93750575856046, 208.06250578986644, 209.18750582117244, 210.31250585247844, 211.43750588378438, 212.56250591509036, 213.68750594639636, 214.8125059777023, 215.93750600900827, 217.06250604031428, 218.18750607162025, 219.31250610292625, 220.43750613423222, 221.5625061655382, 222.68750619684414, 223.8125062281501, 224.93750625945611, 226.0625062907621, 227.18750632206806, 228.31250635337403, 229.43750638468003, 230.562506415986, 231.68750644729198, 232.81250647859798, 233.93750650990395, 235.06250654120993, 236.1875065725159, 237.31250660382187, 238.43750663512785, 239.56250666643382, 240.68750669773982, 241.8125067290458, 242.93750676035177, 244.06250679165774, 245.1875068229637, 246.31250685426969, 247.43750688557566, 248.56250691688166, 249.68750694818763, 250.8125069794936, 251.93750701079958, 253.06250704210555, 254.18750707341152, 255.3125071047175, 256.4375071360235, 257.56250716732944, 258.68750719863544, 259.81250722994145, 260.9375072612474, 262.0625072925534, 263.18750732385934, 264.31250735516534, 265.4375073864713, 266.5625074177773, 267.6875074490833, 268.81250748038923, 269.93750751169523, 271.0625075430012, 272.1875075743072, 273.3125076056131, 274.4375076369191, 275.5625076682251, 276.68750769953107, 277.81250773083707, 278.937507762143]
# depth_array = [18291.69907869359, 18242.83482882235, 18114.866813380635, 17898.18237071001, 17640.554201669125, 17431.229914956257, 17363.636121788822, 17407.20559762794, 17390.722394519336, 17296.74476868623, 17174.482637010617, 17149.53348891504, 17168.654992247943, 17180.589403903, 17200.522059161845, 17217.218023331243, 17204.048371047917, 17163.439259861054, 17123.011382648383, 17098.50324355691, 17087.638511087433, 17080.217310469212, 17069.92090078294, 17052.758994161115, 17028.540510070477, 17000.25368108382, 16969.354575717254, 16937.775706450604, 16905.542269691883, 16870.482840908986, 16833.017148312443, 16792.8983307547, 16749.33978784947, 16702.795075859056, 16653.84533814568, 16601.099731416223, 16543.987609126863, 16483.582388847575, 16419.78280597058, 16352.0179510627, 16280.621591226887, 16206.763498989732, 16131.367668014323, 16052.242382731776, 15969.939829222394, 15883.960766245365, 15793.8288474344, 15700.0262126678, 15601.400222666138, 15497.92657772884, 15390.943357126725, 15279.64652596148, 15164.403828470417, 15045.189362880914, 14922.405878639316, 14794.409383611788, 14660.201557736642, 14522.813155398495, 14380.899455928437, 14234.061430542197, 14083.339257184518, 13933.512981572987, 13781.321059221471, 13624.640691856772, 13465.930422267842, 13300.975405922161, 13137.394379604017, 12967.010239979872, 12791.689026573798, 12615.014341587974, 12435.995245629107, 12256.823944449528, 12077.56768926381, 11895.803025308373, 11714.804102177119, 11530.061998919029, 11339.13865144044, 11150.82557322146, 10958.573632359996, 10768.67033703496, 10579.041643536464, 10392.683541471846, 10199.621511096342, 10010.526869366584, 9822.925276180467, 9634.473834972156, 9444.347207319333, 9251.870037116343, 9059.504108633279, 8871.96619164751, 8682.512614011002, 8502.102273046203, 8318.146398078541, 8149.154405711609, 7971.571146827001, 7809.22410175491, 7634.575217409085, 7472.7915587113475, 7316.287793998056, 7163.931060573051, 7012.134925653768, 6877.188234959844, 6730.091755807167, 6607.906018878991, 6461.290850234716, 6332.514001133509, 6214.391969410131, 6092.369993365461, 5994.642990440025, 5876.727124698508, 5779.247453957066, 5663.790328585504, 5562.30805263055, 5428.440145592694, 5255.73074536742, 5167.387817061854, 5383.72528771256, 5308.710918349109, 5229.280696412716, 5196.927660188645, 5218.47921322444, 5264.6727547731125, 5361.059255371503, 5515.319301799851]

# LENS
lats = [-0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.13356644039944762, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476, -0.1335664403994476]
lons = [140.56250391150786, 141.68750394281383, 142.8125039741198, 143.93750400542578, 145.06250403673175, 146.18750406803773, 147.3125040993437, 148.4375041306497, 149.56250416195567, 150.68750419326165, 151.81250422456762, 152.9375042558736, 154.06250428717956, 155.18750431848554, 156.31250434979154, 157.4375043810975, 158.56250441240348, 159.68750444370946, 160.81250447501543, 161.9375045063214, 163.06250453762738, 164.18750456893338, 165.31250460023935, 166.43750463154532, 167.5625046628513, 168.68750469415727, 169.81250472546324, 170.93750475676921, 172.06250478807522, 173.1875048193812, 174.31250485068716, 175.43750488199314, 176.5625049132991, 177.68750494460508, 178.81250497591105, 179.93750500721706, 181.062505038523, 182.18750506982897, 183.31250510113495, 184.43750513244092, 185.5625051637469, 186.6875051950529, 187.81250522635887, 188.93750525766484, 190.0625052889708, 191.18750532027678, 192.31250535158276, 193.43750538288876, 194.56250541419473, 195.68750544550073, 196.81250547680668, 197.93750550811265, 199.06250553941862, 200.1875055707246, 201.3125056020306, 202.4375056333366, 203.56250566464254, 204.68750569594854, 205.8125057272545, 206.93750575856046, 208.06250578986644, 209.18750582117244, 210.31250585247844, 211.43750588378438, 212.56250591509036, 213.68750594639636, 214.8125059777023, 215.93750600900827, 217.06250604031428, 218.18750607162025, 219.31250610292625, 220.43750613423222, 221.5625061655382, 222.68750619684414, 223.8125062281501, 224.93750625945611, 226.0625062907621, 227.1875063220681, 228.31250635337403, 229.43750638468003, 230.562506415986, 231.68750644729198, 232.81250647859798, 233.93750650990395, 235.06250654120993, 236.1875065725159, 237.31250660382187, 238.43750663512785, 239.56250666643382, 240.68750669773982, 241.8125067290458, 242.93750676035177, 244.06250679165774, 245.1875068229637, 246.31250685426969, 247.43750688557566, 248.56250691688166, 249.68750694818763, 250.8125069794936, 251.93750701079958, 253.06250704210555, 254.18750707341152, 255.3125071047175, 256.4375071360235, 257.56250716732944, 258.68750719863544, 259.81250722994145, 260.9375072612474, 262.0625072925534, 263.18750732385934, 264.31250735516534, 265.4375073864713, 266.5625074177773, 267.6875074490833, 268.81250748038923, 269.93750751169523, 271.0625075430012, 272.1875075743072, 273.3125076056131, 274.4375076369191, 275.5625076682251, 276.68750769953107, 277.81250773083707, 278.937507762143]
depth_array = [17789.362771671138, 17701.38634325639, 17566.123569095045, 17393.913720149994, 17229.54294417893, 17113.266122543915, 17099.2241049467, 17163.508645171856, 17183.089304879883, 17123.490512232962, 17031.50415269238, 16995.608421045814, 16960.58062495017, 16916.205530961808, 16899.35629479993, 16898.5339866589, 16890.625097961434, 16873.906297391284, 16855.083433439504, 16836.336623837902, 16819.192570773674, 16804.33401426953, 16789.525519489176, 16771.202911805864, 16748.78120227195, 16723.962353211267, 16697.204174252875, 16668.32360190764, 16638.400286030148, 16606.149543661882, 16571.50688955113, 16535.8192307252, 16497.554329051938, 16456.82782024431, 16414.84612241767, 16369.802500029346, 16323.61907922522, 16274.013726714003, 16222.215922805364, 16167.846354602394, 16109.993401522495, 16049.853348322937, 15986.30446522046, 15919.923571892281, 15849.376582590767, 15776.70105636992, 15700.234305646862, 15621.275875006793, 15539.183407428849, 15453.830847614361, 15366.98988792038, 15275.611993714643, 15181.17598477818, 15083.829497337743, 14981.944221334164, 14875.930216268429, 14767.475821306067, 14654.599789802694, 14537.724210122256, 14415.591806511893, 14289.196656750612, 14160.083591109083, 14024.263087280695, 13886.227437045647, 13743.399655459367, 13598.970381680512, 13447.9632815425, 13293.208583077288, 13132.808016611065, 12972.137569234936, 12799.80253410355, 12626.201976286404, 12454.370479975527, 12274.041104343154, 12092.662241543752, 11898.359008121872, 11706.630643318475, 11515.79529037559, 11317.021447823097, 11114.272399967267, 10912.671689760098, 10702.178231724642, 10493.101469719859, 10284.232097069245, 10067.889052662551, 9852.249397739917, 9637.951985862135, 9417.587506954442, 9204.888234515343, 8981.140260272217, 8770.242195369914, 8554.88335685416, 8347.519129272156, 8133.396067383759, 7922.961890539604, 7711.138312732172, 7515.994496210777, 7317.243592685338, 7127.518975279742, 6938.827708425728, 6751.0620426496, 6576.399020374578, 6405.0480328408985, 6232.950335425124, 6073.1372096068035, 5919.435886494735, 5786.949690988215, 5649.877875099506, 5539.843297135235, 5429.8964684677385, 5336.7556551572, 5237.592693373112, 5176.205130632138, 5031.694041134714, 4793.76703011523, 4795.897636084995, 4986.992219544471, 4959.300259807436, 4919.154952176306, 4931.862780002624, 4983.158558384578, 5055.610309238453, 5162.049150236948, 5324.379054592556]

# depth_min, depth_max = 5000, 18000  # in meters
# depth_array = np.interp(lons, [lon_min, lon_max], [depth_max, depth_min])

pset = ParticleSet.from_list(fieldset=fieldset, 
                            pclass=JITParticle,
                            lon=lons,
                            lat=lats,
                            depth=depth_array)

In [17]:
output_file = pset.ParticleFile(
    name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens{}_EQ_startingat5m_1958_1977".format(ENS_MEMB), 
                               outputdt=timedelta(days=30))

In [18]:
# Run the particle simulation
print("Starting particle simulation...")
pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],                    # 4th-order Runge-Kutta advection
             runtime=timedelta(days=365*20),      # Run for 7 days
             dt=timedelta(days=-2),       # Time step of 2 days
             output_file=output_file,        # Save trajectories
             
            )

print("Simulation completed!")

Starting particle simulation...
INFO: Output files are stored in /glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens90_EQ_startingat5m_1958_1977.zarr.
100%|██████████| 630720000.0/630720000.0 [13:31<00:00, 777404.76it/s] 
Simulation completed!


# Run Parcels

In [34]:
output_file = pset.ParticleFile(
    name="/glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens85_NH_startingat5m_1958_1977", 
                               outputdt=timedelta(days=30))

In [35]:
# Run the particle simulation
print("Starting particle simulation...")
pset.execute([AdvectionRK4_3D, CheckOutOfBounds, KeepInOcean],                    # 4th-order Runge-Kutta advection
             runtime=timedelta(days=365*20),      # Run for 7 days
             dt=timedelta(days=2),       # Time step of 2 days
             output_file=output_file,        # Save trajectories
             
            )

print("Simulation completed!")

Starting particle simulation...
INFO: Output files are stored in /glade/derecho/scratch/cassiacai/particle_trajectories/particle_trajectories_lens65_NH_startingat5m_1958_1977.zarr.
100%|██████████| 630720000.0/630720000.0 [12:39<00:00, 830844.98it/s] 
Simulation completed!
